In [1]:
from typing import Any, Dict
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, UserMessage
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from src.requestcompletion.nodes.library import from_function

## Travel Planner Agent Demo

We will make a travel planning agent that can help you plan your next trip. We will make a NotionAgent and provide the notion agent as a tool to the travel planning agent. 

First, we will create a NotionAgent that can help us create a new page, add a block to it, find a page, and find and tag a user.

### Step 1: Creating the NotionAgentTool node

In [2]:
# Importing all functions that the notion_agent can invoke
from demo.sample_tools.notion_tools import find_page, create_page, add_block, find_user, tag_user

Making a tool_calling_llm using the inherited ***class based implementation*** for demo. 

We can use easy_wrappers as well (further down)

In [3]:
class NotionAgenticTool(rc.library.ToolCallLLM):
    def __init__(self, message_history: MessageHistory):
        super().__init__(message_history=message_history, 
                         model=rc.llm.OpenAILLM("gpt-4o"),  # or any other model you want to use
                         )

    # define a set of tools that the agent can use
    def connected_nodes(self):  
        return {
            from_function(find_page),   # make a node from a function in one line 🥰
            from_function(create_page),
            from_function(add_block),
            from_function(find_user),
            from_function(tag_user),
            }
    
    # =================== If this node is a tool, then it needs to implement the following ==================
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="NotionAgenticTool",
            detail="A notion tool that can find pages, create pages, add blocks, find users, and tag users.",
            parameters={rc.llm.Parameter(name="notion_task_request", param_type="string", description="The detailed task prompt for the task to be completed.")},
        )

    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]):
        message_hist = MessageHistory(
            [UserMessage(f"Request Prompt: '{tool_parameters['notion_task_request']}'")]
        )
        return cls(message_hist)
    # ===========================================================================================================

    @classmethod
    def pretty_name(cls) -> str:
        return "Notion Agentic Tool"


### Step 2: Make the TravelAgent tool_calling node and give it all the tools

In [4]:
# Import (or write) the functions that we want to give to the agent as tools
from demo.sample_tools.travel_planning_tools import available_locations, currency_used, average_location_cost, convert_currency
convert_currency_node = rc.library.from_function(convert_currency)
available_locations_node = rc.library.from_function(available_locations)
currency_used_node = rc.library.from_function(currency_used)
average_location_cost_node = rc.library.from_function(average_location_cost)    

Making an Agent (tool_calling_llm) using easy_usage_wrapper

In [5]:
TravelAgent = rc.library.tool_call_llm(
    connected_nodes={
        convert_currency_node,
        available_locations_node,
        currency_used_node,
        average_location_cost_node,
        NotionAgenticTool,
    },
    pretty_name="Travel Agent",
    system_message=rc.llm.SystemMessage(
        "You are a travel planner that will plan a trip. you have access to AvailableLocations, ConvertCurrency, CurrencyUsed, AverageLocationCost and NotionAgenticTool tools. Use them when you need to."
    ),
    model=rc.llm.OpenAILLM("gpt-4o"),  # you can use any model you want
)

### Step 3: Invoke the Agent with the User Input

In [ ]:
USER_PROMPT = "I live in Vancouver. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in CAD." \
"Please make a notion page called Vancouver Travel Summary under the page 'Agent Demo Root' and write my travel summary in it."

In [7]:
with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(TravelAgent, message_history=MessageHistory([UserMessage(USER_PROMPT)]))

[+2.822  s] RC.RUNNER   : INFO     - START CREATED Travel Agent - (, message_history=user: I live in Vancouver. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in CAD.Please make a notion page called Aryan Travel Summary under the page 'Agent Demo Root' and write my travel summary in it.)
[+5.336  s] RC.RUNNER   : INFO     - Travel Agent CREATED currency_used Node - ({'location': 'Denmark'}, )
[+5.336  s] RC.RUNNER   : INFO     - currency_used Node DONE EUR
[+8.629  s] RC.RUNNER   : INFO     - Travel Agent CREATED currency_used Node - ({'location': 'Germany'}, )
[+8.630  s] RC.RUNNER   : INFO     - currency_used Node DONE EUR
[+11.889 s] RC.RUNNER   : INFO     - Travel Agent CREATED currency_used Node - ({'location': 'New York'}, )
[+11.889 s] RC.RUNNER   : INFO     - currency_used Node DONE USD
[+13.517 s] RC.RUNNER   : INFO     - Travel Agent CREATED average_location_cost 

### Additional: Graphical representation of the nodes and the flow of data

In [8]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()

# Extended Demo: Vocal Travel Planner Agent

To the agent that we constructed above, we will add an AudioAgent that can record the user's voice and transcribe it.

We can then invoke this new VocalTravelAgent to listen to the user's voice and make travel itineraries on Notion.

### Step 1: Import the AudioAgent

In [9]:
from demo.sample_agents import AudioAgent

### Step 2: Create the VocalTravelAgent by invoking the AudioAgent and TravelAgent in a sequencial manner.

In [10]:
async def vocal_travel_planner(user_prompt: str):
    voice_transcription = await rc.call(AudioAgent, 
                           message_history=MessageHistory([UserMessage(user_prompt)]))
    
    await rc.call(TravelAgent, message_history=MessageHistory([UserMessage(voice_transcription.user_voice_command)]))
    
VocalTravelAgent = from_function(vocal_travel_planner)

### Step 3: Invoke the Agent and give it a voice command

Recorded audio file: demo/audio_recordings/travel_plan.wav

Recorded audio content: 

*I am planning a trip from NY to Germany. I will stay in Germany for 3 days followed by Vancouver for a week. Give me an estimated budget for this trip in USD and put it under the notion page Travel Summary.*

In [11]:
# use incase there is too much background noise
USER_PROMPT = "I have recorded my voice command in demo/audio_recordings/travel_plan.wav. Please transcribe it."
# USER_PROMPT = "Record my audio for 15 seconds and transcribe it."

In [12]:
with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(VocalTravelAgent,
                              user_prompt=USER_PROMPT,
                              )

[+34.209 s] RC.RUNNER   : INFO     - START CREATED vocal_travel_planner Node - (, user_prompt=I have recorded my voice command in demo/audio_recordings/travel_plan.wav. Please transcribe it.)
[+34.209 s] RC.RUNNER   : INFO     - START CREATED vocal_travel_planner Node - (, user_prompt=I have recorded my voice command in demo/audio_recordings/travel_plan.wav. Please transcribe it.)
[+34.209 s] RC.RUNNER   : INFO     - vocal_travel_planner Node CREATED Audio Agent - (, message_history=user: I have recorded my voice command in demo/audio_recordings/travel_plan.wav. Please transcribe it.)
[+34.209 s] RC.RUNNER   : INFO     - vocal_travel_planner Node CREATED Audio Agent - (, message_history=user: I have recorded my voice command in demo/audio_recordings/travel_plan.wav. Please transcribe it.)
[+35.842 s] RC.RUNNER   : INFO     - Audio Agent CREATED transcribe_audio Node - ({'filename': 'demo/audio_recordings/travel_plan.wav'}, )
[+35.842 s] RC.RUNNER   : INFO     - Audio Agent CREATED tran

### Additional: Graph Visualization

In [13]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()